# Implementation of value iteration algorithm

This is a simple 4 x 4 environment, and each block represents a state. The agent can move left, right, up, or down from a state. The "intended" outcome occurs with probability 0.6, but with probability 0.2 the agent moves at right angles to the intended direction. A collision with the wall or boundary results in no movement. The two terminal states have reward -2 and 2, respectively, and all other states have a constant reward of -0.1.

Implementing Value iternation:

In [1]:
# Arguments
REWARD = -0.1 # constant reward for non-terminal states
DISCOUNT = 0.7
MAX_ERROR = 10**(-3)

In [2]:
import random

In [3]:
# Set up the initial environment
NUM_ACTIONS = 4
ACTIONS = [(1, 0), (0, -1), (-1, 0), (0, 1)] # Down, Left, Up, Right
NUM_ROW = 4
NUM_COL = 4
U = [[0, 0, 0, -2], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 2]]

In [4]:
# Visualization
def printEnvironment(arr, policy=False):
    res = ""
    for r in range(NUM_ROW):
        res += "|"
        for c in range(NUM_COL):
            if r ==1 and c == 2:
                val = "WALL"
            elif r ==0  and c == 3:
                val = "-2"    
            elif r == 3 and c==3:
                val = "2"
            else:
                if policy:
                    val = ["Down", "Left", "Up", "Right"][arr[r][c]]
                else:
                    val = str(arr[r][c])
            res += " " + val[:5].ljust(5) + " |" # format
        res += "\n"
    print(res)

In [5]:
# Get the utility of the state reached by performing the given action from the given state
def getU(U, r, c, action):
    dr, dc = ACTIONS[action]
    newR, newC = r+dr, c+dc
    if newR < 0 or newC < 0 or newR >= NUM_ROW or newC >= NUM_COL or (newR == newC == 1): # collide with the boundary or the wall
        return U[r][c]
    else:
        return U[newR][newC]

In [6]:
# Calculate the utility of a state given an action
def calculateU(U, r, c, action):
    u = REWARD
    u += 0.2 * DISCOUNT * getU(U, r, c, (action-1)%4)
    u += 0.6 * DISCOUNT * getU(U, r, c, action)
    u += 0.2 * DISCOUNT * getU(U, r, c, (action+1)%4)
    return u

In [7]:
def valueIteration(U):
    print("During the value iteration:\n")
    while True:
        nextU = [[0, 0, 0, -2], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 2]]
        error = 0
        for r in range(NUM_ROW):
            for c in range(NUM_COL):
                if (r == 0 and c == 3) or (r == 3 and c == 3) or (r == 1 and c == 2):
                    continue
                nextU[r][c] = max([calculateU(U, r, c, action) for action in range(NUM_ACTIONS)]) # Bellman update
                error = max(error, abs(nextU[r][c]-U[r][c]))
        U = nextU
        printEnvironment(U)
        if error < MAX_ERROR * (1-DISCOUNT) / DISCOUNT:
            break
    return U

In [8]:
# Get the optimal policy from U
def getOptimalPolicy(U):
    policy = [[-1, -1, -1, -1] for i in range(NUM_ROW)]
    for r in range(NUM_ROW):
        for c in range(NUM_COL):
            if (r == 0 and c == 3) or (r == 3 and c == 3) or (r == 1 and c == 2):
                continue
            # Choose the action that maximizes the utility
            maxAction, maxU = None, -float("inf")
            for action in range(NUM_ACTIONS):
                u = calculateU(U, r, c, action)
                if u > maxU:
                    maxAction, maxU = action, u
            policy[r][c] = maxAction
    return policy

In [9]:
# Print the initial environment
print("The initial U is:\n")
printEnvironment(U)

The initial U is:

| 0     | 0     | 0     | -2    |
| 0     | 0     | WALL  | 0     |
| 0     | 0     | 0     | 0     |
| 0     | 0     | 0     | 2     |



Convergence:

In [10]:
# Value iteration
U = valueIteration(U)

During the value iteration:

| -0.1  | -0.1  | -0.1  | -2    |
| -0.1  | -0.1  | WALL  | -0.1  |
| -0.1  | -0.1  | -0.1  | 0.74  |
| -0.1  | -0.1  | 0.74  | 2     |

| -0.16 | -0.16 | -0.15 | -2    |
| -0.16 | -0.12 | WALL  | 0.196 |
| -0.16 | -0.16 | 0.314 | 0.829 |
| -0.16 | 0.182 | 0.829 | 2     |

| -0.21 | -0.21 | -0.19 | -2    |
| -0.21 | -0.14 | WALL  | 0.275 |
| -0.21 | 0.033 | 0.364 | 0.900 |
| -0.07 | 0.250 | 0.900 | 2     |

| -0.25 | -0.24 | -0.21 | -2    |
| -0.25 | -0.11 | WALL  | 0.316 |
| -0.12 | 0.092 | 0.408 | 0.917 |
| -0.03 | 0.317 | 0.917 | 2     |

| -0.27 | -0.25 | -0.23 | -2    |
| -0.22 | -0.09 | WALL  | 0.329 |
| -0.10 | 0.129 | 0.426 | 0.925 |
| 0.010 | 0.342 | 0.925 | 2     |

| -0.26 | -0.26 | -0.24 | -2    |
| -0.20 | -0.07 | WALL  | 0.334 |
| -0.07 | 0.145 | 0.436 | 0.929 |
| 0.031 | 0.354 | 0.929 | 2     |

| -0.26 | -0.27 | -0.24 | -2    |
| -0.18 | -0.06 | WALL  | 0.337 |
| -0.06 | 0.153 | 0.440 | 0.931 |
| 0.042 | 0.360 | 0.931 | 2     |

| -0.25 | -0

Optimal policy:

In [11]:
# Get the optimal policy from U and print it
policy = getOptimalPolicy(U)
print("The optimal policy is:\n")
printEnvironment(policy, True)

The optimal policy is:

| Down  | Left  | Left  | -2    |
| Down  | Down  | WALL  | Down  |
| Right | Right | Down  | Down  |
| Right | Right | Right | 2     |

